In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns

In [ ]:
real = pd.read_csv('True.csv')
fake = pd.read_csv('Fake.csv')

In [ ]:
real['is_real'] = 1
fake['is_real'] = 0

In [ ]:
data = pd.concat([real, fake], ignore_index=True)

In [ ]:
data.dropna(inplace=True, axis=1)

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.head()

In [ ]:
data['text'] = data['title'] + ' ' + data['text']
data.drop('title', axis=1, inplace=True)

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

In [ ]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub('https?://\S+|www\.\S+', 'URL', text)
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    text = re.sub('\\W', ' ', text)
    text = re.sub('\n', '', text)
    text = text.strip() # haal spaties weg
    return text
#     words = word_tokenize(text) # 'ik ben ali' -> ['ik', 'ben', 'ali']
#     return ' '.join(lemmatizer.lemmatize(word) for word in words if word not in stop_words)
    
#     text = re.sub("[^a-zA-Z0-9/-]+"," ", text)
#     text = text.strip() # haal spaties weg

#     words = word_tokenize(text) # 'ik ben ali' -> ['ik', 'ben', 'ali']

#     #Removing the stopwords from text
#     new_text = []

#     for word in words:
#         if word not in stop_words:
#             word = word.strip() # haal spaties aan uiteinde weg
#             word = lemmatizer.lemmatize(word) # rocks -> rock, better -> good, running -> run
#             new_text.append(word)

    return text 


In [ ]:
data['text'] = data['text'].apply(clean_text)

In [ ]:
data['text'][1]

In [ ]:
X = data['text']
y = data['is_real']

---

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_train[0]

In [ ]:
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test  = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=250)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding="post", maxlen=250)

In [ ]:
from keras.layers import SimpleRNN,Embedding, Dense, Bidirectional, LSTM, Dense, Dropout
from keras.models import Sequential

In [ ]:
model = Sequential()

model.add(Embedding(max_words, 128))
model.add(SimpleRNN(64, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_split=0.1, batch_size=64, shuffle=True)

In [ ]:
model = Sequential()
model.add(Embedding(max_words, 128))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(16)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_split=0.1, batch_size=64, shuffle=True)